In [1]:
import numpy as np
import pandas as pd
from dwave.system import DWaveSampler, EmbeddingComposite
import dimod
import dwave.inspector as ins
from pyqubo import Spin

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros del modelo
params = {
    'size': 5,  # Tamaño del tablero.
    'chain_strength': 2.0  # Fuerza de la cadena para embedding
}

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): Spin(f's_{i}_{j}') for i in range(params['size']) for j in range(params['size'])}

# Hamiltoniano cuántico adaptado
def create_quantum_hamiltonian(i, j):
    coeffs = [1.0] * 12
    neighbors = [
        (i-1, j) if i > 0 else None,
        (i+1, j) if i < params['size']-1 else None,
        (i, j-1) if j > 0 else None,
        (i, j+1) if j < params['size']-1 else None
    ]
    valid_neighbors = [spins[n] for n in neighbors if n is not None]
    
    H_quantum = (
        coeffs[0] * spins[(i, j)] * valid_neighbors[0] +
        coeffs[1] * spins[(i, j)] * valid_neighbors[1] +
        coeffs[2] * spins[(i, j)] * valid_neighbors[2] +
        coeffs[3] * spins[(i, j)] * valid_neighbors[3] if len(valid_neighbors) > 3 else 0
    )
    return H_quantum

# Construir el Hamiltoniano cuántico total
H = 0
for i in range(params['size']):
    for j in range(params['size']):
        H += create_quantum_hamiltonian(i, j)

# Compilar el modelo
model = H.compile()

# Convertir a QUBO
qubo, offset = model.to_qubo()

# Imprimir QUBO
print("QUBO:", qubo)

# Crear el modelo binario para la solución de D-Wave
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo, offset=offset)

# Crear un sampler
sampler = EmbeddingComposite(solver)

# Muestrear el modelo
response = sampler.sample(bqm, chain_strength=params['chain_strength'], num_reads=100)

# Convertir la respuesta a un DataFrame de Pandas
df = pd.DataFrame(response.data())

# Imprimir todo el DataFrame
print(df)

# Imprimir la solución con la menor energía
print(response.first)

# Mostrar el inspector y proporcionar el enlace
inspector_url = ins.show(response)
print(f"Inspector URL: {inspector_url}")

# Obtener la primera muestra
first_sample = response.first.sample

# Transformar resultados QUBO a representación Ising
board_state = np.zeros((params['size'], params['size']))
for (i, j), spin in spins.items():
    q_value = first_sample.get(f's_{i}_{j}', 0)
    s_value = 2 * q_value - 1  # Convertir de QUBO a Ising
    board_state[i, j] = s_value

print(board_state)


QUBO: {('s_0_1', 's_1_1'): 4.0, ('s_1_1', 's_2_1'): 8.0, ('s_1_0', 's_1_1'): 4.0, ('s_1_1', 's_1_2'): 8.0, ('s_0_2', 's_1_2'): 4.0, ('s_2_1', 's_2_2'): 8.0, ('s_1_2', 's_2_2'): 8.0, ('s_1_2', 's_1_3'): 8.0, ('s_0_3', 's_1_3'): 4.0, ('s_2_2', 's_2_3'): 8.0, ('s_1_3', 's_2_3'): 8.0, ('s_1_3', 's_1_4'): 4.0, ('s_2_1', 's_3_1'): 8.0, ('s_2_0', 's_2_1'): 4.0, ('s_2_2', 's_3_2'): 8.0, ('s_3_1', 's_3_2'): 8.0, ('s_2_3', 's_3_3'): 8.0, ('s_3_2', 's_3_3'): 8.0, ('s_2_3', 's_2_4'): 4.0, ('s_3_1', 's_4_1'): 4.0, ('s_3_0', 's_3_1'): 4.0, ('s_3_2', 's_4_2'): 4.0, ('s_3_3', 's_4_3'): 4.0, ('s_3_3', 's_3_4'): 4.0, ('s_0_1', 's_0_1'): -2.0, ('s_1_1', 's_1_1'): -12.0, ('s_2_1', 's_2_1'): -14.0, ('s_1_0', 's_1_0'): -2.0, ('s_1_2', 's_1_2'): -14.0, ('s_0_2', 's_0_2'): -2.0, ('s_2_2', 's_2_2'): -16.0, ('s_1_3', 's_1_3'): -12.0, ('s_0_3', 's_0_3'): -2.0, ('s_2_3', 's_2_3'): -14.0, ('s_1_4', 's_1_4'): -2.0, ('s_3_1', 's_3_1'): -12.0, ('s_2_0', 's_2_0'): -2.0, ('s_3_2', 's_3_2'): -14.0, ('s_3_3', 's_3_3'): -

Serving Inspector on http://127.0.0.1:18001/?problemId=d76874c3-9dd4-43a6-ab38-1d9d3753c786

Inspector URL: http://127.0.0.1:18001/?problemId=d76874c3-9dd4-43a6-ab38-1d9d3753c786
[[-1. -1.  1. -1. -1.]
 [-1.  1. -1.  1. -1.]
 [ 1. -1.  1. -1.  1.]
 [-1.  1. -1.  1. -1.]
 [-1. -1.  1. -1. -1.]]


In [ ]:
# Importa la biblioteca sgf para manejar archivos SGF y plotly.graph_objects para visualización.
import sgf
import plotly.graph_objects as go

# Función para obtener las posiciones de los vecinos de una celda en un tablero.
def get_neighbors_positions(i, j):
    # Devuelve una lista de tuplas con las posiciones de los vecinos (arriba, abajo, izquierda, derecha).
    return [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]

# Función para obtener los vecinos de una posición específica en el tablero.
def get_neighbors(board, i, j):
    # Obtiene las posiciones de los vecinos.
    neighbors_pos = get_neighbors_positions(i, j)
    # Filtra y devuelve los valores de los vecinos que están dentro de los límites del tablero.
    neighbors = [board[x][y] for x, y in neighbors_pos if x >= 0 and y >= 0 and x < len(board[0]) and y < len(board[0])]
    return neighbors

# Función para extraer movimientos y comentarios de un archivo SGF.
def extract_moves_and_comments_from_sgf(file_path):
    # Abre y lee el contenido del archivo SGF.
    with open(file_path, 'r', encoding='utf-8') as f:
        sgf_content = f.read()

    # Parsea el contenido SGF.
    collection = sgf.parse(sgf_content)
    game = collection[0]

    moves = []  # Lista para almacenar los movimientos.
    actual_winner = None  # Variable para almacenar el ganador del juego.

    # Extrae el resultado del juego para determinar el ganador.
    if 'RE' in game.root.properties:
        result = game.root.properties['RE'][0]
        if result.startswith('B'):
            actual_winner = 'B'  # Ganador: Negro
        elif result.startswith('W'):
            actual_winner = 'W'  # Ganador: Blanco
        # Se pueden añadir más condiciones para manejar 'draw' o 'no result'.

    # Itera sobre los nodos del juego para extraer los movimientos.
    for node in game.rest:
        move_entry = {}
        if 'B' in node.properties:
            move_entry['color'] = 'B'
            move_entry['coords'] = node.properties['B'][0]
            if 'C' in node.properties:
                move_entry['comment'] = node.properties['C'][0]
        elif 'W' in node.properties:
            move_entry['color'] = 'W'
            move_entry['coords'] = node.properties['W'][0]
            if 'C' in node.properties:
                move_entry['comment'] = node.properties['C'][0]

        if move_entry:  # Si hay un movimiento, añádelo a la lista.
            moves.append(move_entry)

    # Devuelve la lista de movimientos y el ganador del juego.
    return moves, actual_winner

# La dirección local de tu archivo SGF.
file_path = "C:/Users/ometi/Documents/sgf partidas/3a0a-gokifu-20210512-Park_Jungwhan-Zhao_Chenyu.sgf"
# Llama a la función para extraer movimientos y comentarios del archivo SGF.
moves, winner = extract_moves_and_comments_from_sgf(file_path)
# 'moves' es una lista de diccionarios con los movimientos y comentarios.
print(f"Total de movimientos: {len(moves)}")
print(f"Ganador del juego: {winner}")
# Muestra los primeros 5 movimientos y comentarios, si existen.
for move in moves[:5]:  
    print(f"Move: {move.get('color')} at {move.get('coords')}", f"Comment: {move.get('comment')}" if 'comment' in move else "")

In [2]:
import numpy as np
import pandas as pd
from dwave.system import DWaveSampler, EmbeddingComposite
import dimod
import dwave.inspector as ins
from pyqubo import Spin, Array

# Configuración del solver
solver_type = 'Advantage2_prototype2.3'
dwave_token = "DEV-951435db241ce06d7f8f8ce960e5c44c5475f72d"
solver = DWaveSampler(solver=solver_type, token=dwave_token)

# Parámetros del modelo
params = {
    'size': 5,  # Tamaño del tablero.
    'chain_strength': 2.0  # Fuerza de la cadena para embedding
}

# Crear variables de Spin para cada posición en el tablero
spins = {(i, j): Spin(f's_{i}_{j}') for i in range(params['size']) for j in range(params['size'])}

# Hamiltoniano cuántico adaptado con condiciones de frontera
def create_quantum_hamiltonian(i, j):
    coeffs = [0.0, 0.0, 1.0, 1.0, 1.0]
    neighbors = [
        (i-1, j) if i > 0 else (i, j),
        (i+1, j) if i < params['size']-1 else (i, j),
        (i, j-1) if j > 0 else (i, j),
        (i, j+1) if j < params['size']-1 else (i, j)
    ]
    
    H_quantum = (
        coeffs[2] * spins[(i, j)] * spins[neighbors[0]] +  # PauliZ(0) @ PauliZ(1)
        coeffs[3] * spins[(i, j)] * spins[neighbors[1]] +  # PauliZ(0) @ PauliX(1)
        coeffs[4] * spins[(i, j)] * spins[neighbors[2]] +  # PauliX(0) @ PauliZ(1)
        coeffs[2] * spins[(i, j)] * spins[neighbors[3]]    # PauliZ(0) @ PauliZ(1) (simétrico)
    )
    return H_quantum

# Construir el Hamiltoniano cuántico total
H = 0
for i in range(params['size']):
    for j in range(params['size']):
        H += create_quantum_hamiltonian(i, j)

# Compilar el modelo
model = H.compile()

# Convertir a QUBO
qubo, offset = model.to_qubo()

# Imprimir QUBO
print("QUBO:", qubo)

# Crear el modelo binario para la solución de D-Wave
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo, offset=offset)

# Crear un sampler
sampler = EmbeddingComposite(solver)

# Muestrear el modelo
response = sampler.sample(bqm, chain_strength=params['chain_strength'], num_reads=100)

# Convertir la respuesta a un DataFrame de Pandas
df = pd.DataFrame(response.data())

# Imprimir todo el DataFrame
print(df)

# Imprimir la solución con la menor energía
print(response.first)

# Mostrar el inspector y proporcionar el enlace
inspector_url = ins.show(response)
print(f"Inspector URL: {inspector_url}")

# Obtener la primera muestra
first_sample = response.first.sample

# Transformar resultados QUBO a representación Ising
board_state = np.zeros((params['size'], params['size']))
for (i, j), spin in spins.items():
    q_value = first_sample.get(f's_{i}_{j}', 0)
    s_value = 2 * q_value - 1  # Convertir de QUBO a Ising
    board_state[i, j] = s_value

print(board_state)


QUBO: {('s_0_1', 's_1_1'): 8.0, ('s_0_0', 's_0_1'): 8.0, ('s_0_1', 's_0_2'): 8.0, ('s_1_0', 's_1_1'): 8.0, ('s_0_0', 's_1_0'): 8.0, ('s_1_1', 's_1_2'): 8.0, ('s_0_2', 's_1_2'): 8.0, ('s_0_2', 's_0_3'): 8.0, ('s_1_2', 's_1_3'): 8.0, ('s_0_3', 's_1_3'): 8.0, ('s_0_3', 's_0_4'): 8.0, ('s_1_3', 's_1_4'): 8.0, ('s_0_4', 's_1_4'): 8.0, ('s_1_0', 's_2_0'): 8.0, ('s_1_1', 's_2_1'): 8.0, ('s_2_0', 's_2_1'): 8.0, ('s_1_2', 's_2_2'): 8.0, ('s_2_1', 's_2_2'): 8.0, ('s_1_3', 's_2_3'): 8.0, ('s_2_2', 's_2_3'): 8.0, ('s_1_4', 's_2_4'): 8.0, ('s_2_3', 's_2_4'): 8.0, ('s_2_0', 's_3_0'): 8.0, ('s_2_1', 's_3_1'): 8.0, ('s_3_0', 's_3_1'): 8.0, ('s_2_2', 's_3_2'): 8.0, ('s_3_1', 's_3_2'): 8.0, ('s_2_3', 's_3_3'): 8.0, ('s_3_2', 's_3_3'): 8.0, ('s_2_4', 's_3_4'): 8.0, ('s_3_3', 's_3_4'): 8.0, ('s_3_0', 's_4_0'): 8.0, ('s_3_1', 's_4_1'): 8.0, ('s_4_0', 's_4_1'): 8.0, ('s_3_2', 's_4_2'): 8.0, ('s_4_1', 's_4_2'): 8.0, ('s_3_3', 's_4_3'): 8.0, ('s_4_2', 's_4_3'): 8.0, ('s_3_4', 's_4_4'): 8.0, ('s_4_3', 's_4_4')

Serving Inspector on http://127.0.0.1:18001/?problemId=0e3b9324-c516-4ad4-b08f-472f9e272abb

Inspector URL: http://127.0.0.1:18001/?problemId=0e3b9324-c516-4ad4-b08f-472f9e272abb
[[-1.  1. -1.  1. -1.]
 [ 1. -1.  1. -1.  1.]
 [-1.  1. -1.  1. -1.]
 [ 1. -1.  1. -1.  1.]
 [-1.  1. -1.  1. -1.]]
